In [2]:
import os
import glob
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from numpy.random import normal
from PIL import Image
import math


In [3]:
class ImageProcessor:
    def __init__(self, input_directory):
        self.input_directory = input_directory
        self.label_map = {}
        self.scaler = None

    def flatten_image(self):
        # List all folders in the input directory
        folders = [f for f in os.listdir(self.input_directory) if os.path.isdir(os.path.join(self.input_directory, f))]
        print(f"Found folders: {folders}")

        # Initialize arrays for inputs and outputs
        X = []
        y = []

        # Create a mapping from folder names to one-hot encoded labels
        self.label_map = {folder: idx for idx, folder in enumerate(folders)}
        num_classes = len(folders)

        # Process each folder and photo
        for folder in folders:
            folder_path = os.path.join(self.input_directory, folder)

            # Sort the list of photos to ensure consistent order
            photos = sorted([p for p in os.listdir(folder_path) if p.endswith(('.png', '.jpg', '.jpeg'))])

            for photo in photos:
                photo_path = os.path.join(folder_path, photo)
                image = Image.open(photo_path)

                # Convert the image to a numpy array and flatten it
                image_array = np.array(image).flatten()
                X.append(image_array)

                # Create a one-hot encoded label
                label = np.zeros(num_classes)
                label[self.label_map[folder]] = 1
                y.append(label)

        # Convert lists to numpy arrays
        X = np.array(X)
        y = np.array(y)

        return X, y

    class NormalizeImage:
        def __init__(self, data):
            self.data = data

        def transform(self):
            # Normalize data to the range [-1, 1]
            return self.data / 255

        def inverse_transform(self):
            return self.data * 255

    @staticmethod
    def split_train_test(combined_array):
        np.random.shuffle(combined_array)
        # Calculate the split indices
        num_samples = combined_array.shape[0]
        train_end = int(0.7 * num_samples)  # 70% of the data is used for training
        test_end = int(0.85 * num_samples)  # 15% of the data is used for testing

        # Split the data into training, testing, and validation sets
        train_data = combined_array[:train_end]
        test_data = combined_array[train_end:test_end]
        val_data = combined_array[test_end:]

        return train_data, test_data, val_data

    @staticmethod
    def split_input_output(data, num_input_features):
        X_data = data[:, :num_input_features]
        y_data = data[:, num_input_features:]
        return X_data, y_data

    def process_all(self):
        X, y = self.flatten_image()

        self.scaler = self.NormalizeImage(X)
        X_normalized = self.scaler.transform()
        combined_array = np.hstack((X_normalized, y))

        train_data, test_data, val_data = self.split_train_test(combined_array)
        num_input_features = X.shape[1]
        X_train, y_train = self.split_input_output(train_data, num_input_features)
        X_test, y_test = self.split_input_output(test_data, num_input_features)
        X_val, y_val = self.split_input_output(val_data, num_input_features)

        return X_train, y_train, X_test, y_test, X_val, y_val, self.scaler


In [4]:
# Path to the directory containing the dataset
input_directory = "../Dataset/Foto_Resize_Rotate_50x50" 

# Create an instance of the ImageProcessor class
image_processor = ImageProcessor(input_directory)

# Process the data
X_train, y_train, X_test, y_test, X_val, y_val, scaler = image_processor.process_all()

# Display the shape of the datasets
print("Training data:", X_train.shape, y_train.shape)
print("Testing data:", X_test.shape, y_test.shape)
print("Validation data:", X_val.shape, y_val.shape)

# Example of normalized input
print("Normalized training sample:", X_train[0])


Found folders: ['Azmira', 'David', 'Dimas', 'Fadhli', 'Fadlin', 'Hafidz', 'Haidar', 'Hanna', 'Keiko', 'Khansa', 'Mikhael', 'Puti', 'Raesa', 'Satwika', 'Toni']
Training data: (968, 2500) (968, 15)
Testing data: (208, 2500) (208, 15)
Validation data: (208, 2500) (208, 15)
Normalized training sample: [0.68627451 0.68235294 0.68235294 ... 0.70588235 0.69019608 0.69019608]


In [9]:
#50x50
#input = 2500, hidden layer = 1, neuron = 1680
inputLayer = 2500
outputLayer = 15
epoch = 10000
alpha = 0.01
X = X_train
T = y_train
neuron = 10

stddev = np.sqrt(2/inputLayer)


v = np.random.normal(loc=0, scale = stddev, size = (inputLayer, neuron))  # weights from input to hidden layer
print(np.shape(v))

vb = np.random.normal(loc=0, scale = stddev, size = (1, neuron)) # weight biases for hidden layer
print(np.shape(vb))

w = np.random.normal(loc=0, scale = stddev, size = (neuron, outputLayer)) # weights from hidden to output layer
print(np.shape(w))

wb = np.random.normal(loc=0, scale = stddev, size = (1, outputLayer)) # weight biases for output layer
print(np.shape(wb))


(2500, 10)
(1, 10)
(10, 15)
(1, 15)


In [10]:
for e in range(epoch):  # Set the maximum number of epochs
    total_error = 0  # Initialize total error for the epoch

    for i in range(len(X)):
        # Feedforward
        x = X[i]
        t = T[i]  # True label for this sample
        z_in = np.dot(x, v) + vb
        z = 1 / (1 + np.exp(-z_in))  # Hidden layer activation
        y_in = np.dot(z, w) + wb
        y = 1 / (1 + np.exp(-y_in))  # Output layer activation (sigmoid)

        # Flatten activations to avoid shape issues
        z_in = z_in.flatten()
        z = z.flatten()
        y_in = y_in.flatten()
        y = y.flatten()

        # Backpropagation of error
        delta_y = (t - y) * np.exp(-y_in) / ((np.exp(-y_in) + 1) ** 2)
        delta_w = alpha * np.outer(z, delta_y)  # Outer product for weight updates
        delta_wb = alpha * delta_y

        delta_in = np.dot(delta_y, w.T)  # Backpropagated error for hidden layer
        delta = delta_in * np.exp(-z_in) / ((np.exp(-z_in) + 1) ** 2)
        delta_v = alpha * np.outer(x, delta)
        delta_vb = alpha * delta

        # Update weights and biases
        w += delta_w
        wb += delta_wb
        v += delta_v
        vb += delta_vb

        # Calculate cross-entropy loss for this sample
        sample_error = -np.sum(t * np.log(y + 1e-8))
        total_error += sample_error

    # Calculate average error for the epoch
    average_error = total_error / len(X)

    # Print the error every 100 epochs or the last epoch
    if e % 100 == 0 or e == epoch - 1:
        print(f"Epoch {e + 1}: Average Error: {average_error:.4f}")

    # Early stopping condition
    if average_error < 0.01 or e == epoch - 1:
        print(f"Stopping early at epoch {e + 1} with average error: {average_error:.4f}")
        break

Epoch 1: Average Error: 1.7481
Epoch 101: Average Error: 2.6843
Epoch 201: Average Error: 2.3953
Epoch 301: Average Error: 1.6995
Epoch 401: Average Error: 1.1308
Epoch 501: Average Error: 0.9843
Epoch 601: Average Error: 0.8712
Epoch 701: Average Error: 0.7914
Epoch 801: Average Error: 0.7436
Epoch 901: Average Error: 0.7064
Epoch 1001: Average Error: 0.6674
Epoch 1101: Average Error: 0.6223
Epoch 1201: Average Error: 0.5643
Epoch 1301: Average Error: 0.4870
Epoch 1401: Average Error: 0.4138
Epoch 1501: Average Error: 0.3575
Epoch 1601: Average Error: 0.3129
Epoch 1701: Average Error: 0.2815
Epoch 1801: Average Error: 0.2555
Epoch 1901: Average Error: 0.2380
Epoch 2001: Average Error: 0.2211
Epoch 2101: Average Error: 0.2098
Epoch 2201: Average Error: 0.1915
Epoch 2301: Average Error: 0.1881
Epoch 2401: Average Error: 0.1822
Epoch 2501: Average Error: 0.1748
Epoch 2601: Average Error: 0.1685
Epoch 2701: Average Error: 0.1619
Epoch 2801: Average Error: 0.1642
Epoch 2901: Average Error: